In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_lattice as tfl
import numpy as np
import pandas as pd

np.random.seed(1)

In [2]:
# DATA
df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_106_exp_30m_20221130.csv')

In [3]:
df_result = pd.DataFrame({'n_train':[], 'y_max':[], 'grid_size':[], 'lr':[], 'R2':[]})
n_end = 3000

for n_train in ([32, 64, 128]):
    for y_max in ([2, 5, 10, 20]):
        for grid_size in ([3, 10, 20, 50]):
            for lr in ([0.001, 0.01, 0.1, 0.2]):
                
                # DATA
                # trainning set
                Tz_k   = np.array(df['room_temp']).reshape(-1,1)[0: n_train]
                To_k   = np.array(df['thermostat_outside_temp']).reshape(-1,1)[0: n_train]
                Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[1: n_train+1]
                msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[1: n_train+1]
                Tz_k1  = np.array(df['room_temp']).reshape(-1,1)[1: n_train+1]

                X = np.concatenate((Tz_k, Tsa_k1, msa_k1, To_k), axis=1)
                Y = np.copy(Tz_k1)

                # test set
                Tz_k_t   = np.array(df['room_temp']).reshape(-1,1)[n_train: n_end]
                To_k_t   = np.array(df['thermostat_outside_temp']).reshape(-1,1)[n_train: n_end]
                Tsa_k1_t = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+1: n_end+1]
                msa_k1_t = np.array(df['airflow_current']).reshape(-1,1)[n_train+1: n_end+1]
                Tz_k1_t  = np.array(df['room_temp']).reshape(-1,1)[n_train+1: n_end+1]

                X_t = np.concatenate((Tz_k_t, Tsa_k1_t, msa_k1_t, To_k_t), axis=1)
                Y_t = np.copy(Tz_k1_t)

                # MODEL
                # combined calibrator
                combined_calibrators = tfl.layers.ParallelCombination()

                calibrator_x1 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(65,  85,   grid_size), output_min=0, output_max=y_max)
                calibrator_x2 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(50,  100,  grid_size), output_min=0, output_max=y_max, monotonicity='increasing')
                calibrator_x3 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(390, 1010, grid_size), output_min=0, output_max=y_max)
                calibrator_x4 = tfl.layers.PWLCalibration(input_keypoints=np.linspace(35, 51, grid_size), output_min=0, output_max=y_max)

                combined_calibrators.append(calibrator_x1)
                combined_calibrators.append(calibrator_x2)
                combined_calibrators.append(calibrator_x3)
                combined_calibrators.append(calibrator_x4)

                # Lattice
                lattice = tfl.layers.Lattice(
                    lattice_sizes = [y_max+1, y_max+1, y_max+1, y_max+1],
                    monotonicities = ['increasing', 'increasing', 'increasing', 'increasing'],
                    output_min = 65,
                    output_max = 85)
                
                # model
                inps  = tf.keras.Input(shape=(4,))
                calib = combined_calibrators(inps)
                outs  = lattice(calib)

                model = tf.keras.Model(inputs=inps, outputs=outs)
                model.compile(
                    loss = tf.keras.losses.mean_squared_error,
                    optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
                    metrics = [tfa.metrics.RSquare()])
                
                # TRAIN
                callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500)

                hist = model.fit(
                    x = X,
                    y = Y,
                    batch_size = Y.shape[0],
                    epochs = 20000,
                    shuffle = False,
                    validation_data=(X_t, Y_t),
                    callbacks=[callback],
                    verbose = 0)
                
                a = np.argmax(hist.history['val_r_square'])
                df_result.loc[len(df_result)] = [n_train, y_max, grid_size, lr, hist.history['val_r_square'][a]]

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 4,
                       ):
    print(df_result)